# First exploratory steps in cocoa daily prices (2022–2025)

This notebook cleans and structures the **ICCO cocoa daily prices** dataset and prepares it for further analysis.

Main questions we can explore after cleaning:
- Is there evidence of **cost pressure** in cocoa prices (trend / volatility)?
- Are there **seasonal patterns** (monthly/quarterly effects)?
- How do **London vs New York futures** relate to the ICCO daily price?

**Study period for this project:** **2022-01-01 to 2025-12-31**.


# Imports

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv


# Load raw data

In [13]:
DATA_PATH = Path(r"/Users/linaabdulsamad/Desktop/Bootcamp/Chocolate/datasets/jl_cocoa_daily_prices_1994-2025.csv")

# The file is semicolon-separated (;) — important!
raw = pd.read_csv(DATA_PATH, sep=";")

raw.head()

,Date,London futures (£ sterling/tonne),New York futures (US$/tonne),ICCO daily price (US$/tonne),ICCO daily price (Euro/tonne)
0,2025-08-07,5401.33,8116.00,7687.97,6605.12
1,2025-08-06,5404.33,7882.67,7553.05,6487.69
2,2025-08-05,5424.00,7878.33,7551.55,6521.90
3,2025-08-04,5272.33,7638.67,7324.64,6335.45
4,2025-08-01,5384.33,7696.33,7422.56,6419.77


# Clean & structure

In [14]:
df = raw.copy()

# 1) Parse date
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# 2) Standardize column names (optional but recommended)
rename_map = {
    "London futures (£ sterling/tonne)": "london_futures_gbp_per_tonne",
    "New York futures (US$/tonne)": "newyork_futures_usd_per_tonne",
    "ICCO daily price (US$/tonne)": "icco_daily_usd_per_tonne",
    "ICCO daily price (Euro/tonne)": "icco_daily_eur_per_tonne",
}
df = df.rename(columns=rename_map)

# 3) Remove fully empty rows (if any)
df = df.dropna(how="all")

# 4) Remove duplicates (same Date) keeping the last occurrence
df = df.sort_values("Date").drop_duplicates(subset=["Date"], keep="last")

# 5) Filter to study period (2022–2025 inclusive)
start_date = pd.Timestamp("2022-01-01")
end_date = pd.Timestamp("2025-12-31")

df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)].copy()

# 6) Sort and set Date as index (helps time-series analysis)
df = df.sort_values("Date").set_index("Date")

df.head()

,london_futures_gbp_per_tonne,newyork_futures_usd_per_tonne,icco_daily_usd_per_tonne,icco_daily_eur_per_tonne
Date,,,,
2022-01-03,1702.17,2523.67,2406.58,2131.99
2022-01-04,1687.00,2502.67,2392.21,2117.94
2022-01-05,1691.67,2495.67,2394.70,2112.24
2022-01-06,1672.67,2456.33,2358.93,2086.59
2022-01-07,1720.67,2548.33,2441.38,2149.76


# Quick data checks

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 923 entries, 2022-01-03 to 2025-08-07
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   london_futures_gbp_per_tonne   923 non-null    float64
 1   newyork_futures_usd_per_tonne  923 non-null    float64
 2   icco_daily_usd_per_tonne       923 non-null    float64
 3   icco_daily_eur_per_tonne       923 non-null    float64
dtypes: float64(4)
memory usage: 36.1 KB


In [16]:
# Missing values by column
df.isna().sum().sort_values(ascending=False)

london_futures_gbp_per_tonne     0
newyork_futures_usd_per_tonne    0
icco_daily_usd_per_tonne         0
icco_daily_eur_per_tonne         0
dtype: int64

In [17]:
# Basic descriptive stats
df.describe().T

,count,mean,std,min,25%,50%,75%,max
london_futures_gbp_per_tonne,923.0,3947.671636,2193.118525,1672.67,1922.665,3106.33,5672.500,9428.33
newyork_futures_usd_per_tonne,923.0,5123.263629,2766.753834,2121.17,2620.830,3710.33,7661.335,12014.67
icco_daily_usd_per_tonne,923.0,5073.862893,2796.873426,2118.19,2511.655,3750.60,7545.490,11984.66
icco_daily_eur_per_tonne,923.0,4690.852340,2583.651770,2086.59,2319.555,3522.64,6678.700,11442.95


# Optional: tidy missing values

In [18]:
# If you want a continuous daily index (including weekends/holidays), uncomment:
# full_idx = pd.date_range(df.index.min(), df.index.max(), freq="D")
# df_daily = df.reindex(full_idx)

# Recommended approach for price series: forward-fill missing *business days only*,
# but keep transparency by storing a 'was_missing' flag.
df_clean = df.copy()

# Flag rows where any key field is missing
key_cols = [
    "london_futures_gbp_per_tonne",
    "newyork_futures_usd_per_tonne",
    "icco_daily_usd_per_tonne",
    "icco_daily_eur_per_tonne",
]
df_clean["any_missing"] = df_clean[key_cols].isna().any(axis=1)

# Option A (default): keep NaNs (best for honesty)
# Option B: fill short gaps (uncomment if needed)
# df_clean[key_cols] = df_clean[key_cols].interpolate(limit=3)

df_clean.head()

,london_futures_gbp_per_tonne,newyork_futures_usd_per_tonne,icco_daily_usd_per_tonne,icco_daily_eur_per_tonne,any_missing
Date,,,,,
2022-01-03,1702.17,2523.67,2406.58,2131.99,False
2022-01-04,1687.00,2502.67,2392.21,2117.94,False
2022-01-05,1691.67,2495.67,2394.70,2112.24,False
2022-01-06,1672.67,2456.33,2358.93,2086.59,False
2022-01-07,1720.67,2548.33,2441.38,2149.76,False


# Save cleaned data

In [19]:
df_clean.to_csv("/Users/linaabdulsamad/Desktop/Bootcamp/Chocolate/datasets/cocoa_prices_2022_2025_clean.csv", index=True)

In [27]:
from dotenv import dotenv_values
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd
from sqlalchemy import Integer, String, Float, DateTime, Date

config = dotenv_values()

user = config["POSTGRES_USER"]
password = quote_plus(config["POSTGRES_PASS"])
host = config["POSTGRES_HOST"]
port = config["POSTGRES_PORT"]
dbname = config["POSTGRES_DB"]
schema = config["POSTGRES_SCHEMA"]


In [21]:
# Now building the URL with the values from the .env file

url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

In [22]:
schema

'air_force'

In [23]:
df_cocoa_prices = pd.read_csv("/Users/linaabdulsamad/Desktop/Bootcamp/Chocolate/datasets/cocoa_prices_2022_2025_clean.csv", index_col=0, parse_dates=True)

In [24]:
df_cocoa_prices.head()

,london_futures_gbp_per_tonne,newyork_futures_usd_per_tonne,icco_daily_usd_per_tonne,icco_daily_eur_per_tonne,any_missing
Date,,,,,
2022-01-03,1702.17,2523.67,2406.58,2131.99,False
2022-01-04,1687.00,2502.67,2392.21,2117.94,False
2022-01-05,1691.67,2495.67,2394.70,2112.24,False
2022-01-06,1672.67,2456.33,2358.93,2086.59,False
2022-01-07,1720.67,2548.33,2441.38,2149.76,False


In [25]:
df_cocoa_prices.dtypes

london_futures_gbp_per_tonne     float64
newyork_futures_usd_per_tonne    float64
icco_daily_usd_per_tonne         float64
icco_daily_eur_per_tonne         float64
any_missing                         bool
dtype: object

In [31]:
engine = create_engine(url, echo=False)

df_cocoa_prices.to_sql(
    "jl_cocoa_prices_2022_2025", # name of the table in the database
    engine,
    schema = schema,
    if_exists="replace", # this replaces an existing table!
    index=True,
    dtype={
        "Date": Date(),
        "london_futures_gbp_per_tonne": Float(),
        "newyork_futures_usd_per_tonne": Float(),
        "icco_daily_usd_per_tonne": Float(),
        "icco_daily_eur_per_tonne": Float(),
        "any_missing": String(),
    }
)

923